In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Import necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
sns.set(rc={'figure.figsize':(10, 8)}); # you can change this if needed

Load the data. They are located in the input directory.


In [ ]:
df = pd.read_csv('../input/OnlineNewsPopularityReduced.csv')
df.describe()

There are some 0 words articles, they should be deleted

In [ ]:
print((df.n_tokens_content == 0).sum(), 'articles with 0 words')
df[df.n_tokens_content == 0].head()

In [ ]:
df = df[df.n_tokens_content != 0]

In [ ]:
sns.stripplot(df.shares)

Will be better to just delete this one outlier on the right.

In [ ]:
df[df.shares == 284700]

In [ ]:
df = df.drop(584).reset_index(drop=True)

In [ ]:
pd.concat([df.dtypes, df.nunique()], axis=1).transpose()

In [ ]:
df.columns

Adding some useful features.

In [ ]:
df['data_channel_is_none'] = 1 - df[['data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world']].sum(1)
df['data_channel'] = df[['data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'data_channel_is_none']].idxmax(axis=1).str[16:].str.capitalize()
df['weekday'] = df[['weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday']].idxmax(axis=1).str[11:].str.capitalize()
channels = ['Lifestyle',
       'Entertainment', 'Bus',
       'Socmed', 'Tech',
       'World', 'None']
days = ['Monday', 'Tuesday',
       'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday']

**kw_min_min** column has only one value so it can be dropped.

In [ ]:
df.drop(columns='kw_min_min', inplace=True)

#### Read the description of the dataset, define the target variable and the main task of the forecasting model.

Target column is **shares**, which represents number of shares for an article.
Therefore the main task of forecasting model will be to predict how popular will be some news article measured in number of it's shares.

#### On what day of the week were most of the articles in the dataset published? On what day were the fewest articles published? Visualize the information about the days of the week.

In [ ]:
days_count = df.groupby('weekday').weekday.count()[days]
a = days_count.plot(kind='bar')
a.set_xlabel('Day of week')
a.set_ylabel('Count')

We can see that there are lots of articles coming out in weekdays (with Tuesday being top 1) and 2-3 times less in weekends. The fewest articles are published on Saturday.

#### What can you say about the distribution of values of the **n_tokens_title** variable? How is the length of the article title related to the result variable?

In [ ]:
print(df.n_tokens_title.mean(), df.n_tokens_title.var())
df.n_tokens_title.hist(bins=20)

Distribution of **n_token_title** is similar to normal with mean = 11.2 and variance = 4.8

In [ ]:
r = spearmanr(df['shares'], df['n_tokens_title'])
print('Spearman correlation:', r[0], 'p-value:', r[1])
df.plot('n_tokens_title', 'shares', kind='scatter')

In [ ]:
print(df[(df.n_tokens_title < 7) | (df.n_tokens_title > 16)].shape[0])
print(df[df.shares > 10000].shape[0])

Only titles that contain 7-16 words are able to get more than ~10000 shares, but that can be because of low amount of data about other lengthes. Also there are very low Spearman correlation.

#### Which factor has a greater impact on the popularity of an article --- the number of pictures or videos?

In [ ]:
r = spearmanr(df['shares'], df[['num_imgs', 'num_videos']])
print('Spearman correlation with num_imgs:', r[0][0, 1], 'p-value:', r[1][0, 1])
print('Spearman correlation with num_videos:', r[0][0, 2], 'p-value:', r[1][0, 2])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 7), sharey=True)
df.plot('num_imgs', 'shares', kind='scatter', ax=axes[0])
df.plot('num_videos', 'shares', kind='scatter', ax=axes[1])

There are low yet statistically valuable correlation with **num_videos** and decent correlation with **num_img**. Most popular articles (>10000 shares) have small amount of videos and images.

#### Is it true that articles published on weekends are on average more popular than articles published on weekdays? Come up with a visualization.

In [ ]:
df.groupby('is_weekend')['shares'].mean().plot(kind='bar')

In [ ]:
shares_by_day = df.groupby('weekday')['shares'].mean()[days]
print(shares_by_day)
a = shares_by_day.plot(kind='bar')
a.set_ylabel('Mean shares')
a.set_xlabel('Day of week')

Articles indeed are more popular when being published on weekends, especially on Sunday.

#### Is there a relationship between the length of the text (number of words) and the popularity of the article?

This function plots moving average of shares over some feature, it shows how shares tend to change with the increasing of this feature.

In [ ]:
def plot_moving_average(feature, window=100, bins=30, min_periods=25, lims=(None, None), target='shares'):
    d = df[[feature, target]].sort_values(feature)
    d['mavr1'] = d[target].rolling(window=window, min_periods=min_periods).mean()
    d['mavr2'] = d['mavr1'].rolling(window=window, min_periods=min_periods).mean()
    d['mavr3'] = d['mavr2'].rolling(window=window, min_periods=min_periods).mean()
    if lims[0] != None:
        d = d[d[feature] >= lims[0]]
    if lims[1] != None:
        d = d[d[feature] <= lims[1]]
    
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 12), sharex=True)
    axes[0].plot(d[feature], d['mavr3'])
    axes[0].set_xlabel(feature)
    axes[0].set_ylabel('Moving average of ' + target)
    axes[0].set_title(f'Moving average of {target} over the values of {feature}')

    axes[1].hist(d[feature], bins=bins)
    axes[1].set_xlabel('Histogram of ' + feature)

In [ ]:
r = spearmanr(df['shares'], df['n_tokens_content'])
print('Spearman correlation:', r[0], 'p-value:', r[1])
df.plot('n_tokens_content', 'shares', kind='scatter')
plot_moving_average('n_tokens_content')
plot_moving_average('n_tokens_content', lims=(0, 2000))

There are small negative Spearman correlation between these. The charts show that in average, very short articles (< 400 words) have more shares that other.

#### What other insights can be found in the available data? Creative task.

How data channel affects popularity?

In [ ]:
days_count = df.groupby('data_channel').shares.aggregate(['count', 'mean'])
print(days_count)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))
days_count['count'].plot(kind='bar', ax=axes[0])
days_count['mean'].plot(kind='bar', ax=axes[1])
axes[0].set_ylabel('Count')
axes[1].set_ylabel('Mean shares')

Most frequent data channel, world, also has least mean shares.

In [ ]:
data = df.drop(columns=['weekday', 'data_channel', 'url'])

Let's calculate correlations between **shares** and every other column

In [ ]:
corrs = pd.DataFrame(columns=data.columns).transpose()
for c in data.columns:
        corrs.loc[c, 'c'], corrs.loc[c, 'p'] = spearmanr(data['shares'], data[c])
corrs['absc'] = corrs.c.abs()
corrs = corrs.sort_values('absc', ascending=False)

In [ ]:
corrs.head(25)

All of the strongest correlations are also statistically significant, so these parameters will be most important to look at.

Some interesting columns:
- **self\_reference\_\[min, max, avg\]\_shares:** one of the strongest correlations is with average number of shares of referenced articles?
- **global_subjectivity:** the more subjective article is, the more popular it will be
- **rate_positive_words, global_rate_positive_words:** better to use more positive words in the article

#### self_reference_\[min, max, avg\]_shares

In [ ]:
plot_moving_average('self_reference_avg_sharess')
plot_moving_average('self_reference_min_shares')
plot_moving_average('self_reference_max_shares')

In [ ]:
plot_moving_average('self_reference_avg_sharess', lims=(0, 5000))
plot_moving_average('self_reference_min_shares', lims=(0, 5000))
plot_moving_average('self_reference_max_shares', lims=(0, 5000))

There's definitely some connection here, but the fact that the Spearman's coefficient is so large is due to a few particularly popular articles that link to other extremely popular articles

#### global_rate_positive_words

In [ ]:
plot_moving_average('global_rate_positive_words')
plot_moving_average('global_rate_positive_words', lims=(.02, .06))

In the interval with the largest number of values, the larger the **global_rate_positive_words**, the less often the moving average chart falls downward, so it's efficient to use more positive words in articles.

#### global_subjectivity

In [ ]:
plot_moving_average('global_subjectivity')

Surely, the more subjective article is, the more chances it will become popular.

In [ ]:
plot_moving_average('LDA_02', 75)
plot_moving_average('LDA_02', 50, 30, 15, lims=(0, 0.05))

A lot of values of LDA_02 are concentrated in small intervals, so it's hard to inspect them, but for values where LDA_02 > 0.1 there are clear negative correlation with shares.